In [ ]:
%%configure -f
{"defaultLakehouse": {"name": "lh_cfg"}}

# Set variables

In [ ]:
is_debug       = False
medallion_name = "Report"

if is_debug:
    print(f"is_debug:       {is_debug}")
    print(f"medallion_name: {medallion_name}")

# Imports

In [ ]:
import delta.tables                       as dlt
import json                               as j
from   types       import SimpleNamespace as sn
from   pyspark.sql import functions       as sf
from   pyspark.sql import DataFrame       as df
from   pyspark.sql import types           as st
from   inspect     import stack           as stk
import time                               as t
import datetime                           as dt
import pytz                               as pt
import re
import unicodedata                        as ucd
import pandas                             as pd
import sys
import requests                           as req
if is_debug:
  from pyspark.sql import SparkSession    as ss

# Get parameters from "pl_master"

In [ ]:
global_parameter = "global_parameter from pl_master"
workspace_id     = "workspace_id from pl_master"

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

In [ ]:
# Freeze this
# Simulate parameters from "pl_master".

workspace_id = "abc"

global_parameter = """
{
    "days_to_keep_log": 365,
    "send_success_email": "True",
    "send_error_email": "True",
    "email_on_success": "example@example.com",
    "email_on_error": "example@example.com",
    "time_zone_nb": "Canada/Eastern",
    "time_zone_pl": "Eastern Standard Time",
    "mssql_isolation_level": "READ_UNCOMMITTED",
    "projet_documentation_data_file_name": "configuration",
    "sql_server": "sql-server.database.windows.net",
    "abfs_path_lh_log": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "abfs_path_lh_bronze": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "abfs_path_lh_silver": "abfss://abc@onelake.dfs.fabric.microsoft.com/1abc",
    "abfs_path_lh_cfg": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
    "abfs_path_lh_AdventureWorks": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
	"my_dataverse": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc",
	"now_datetime": "2024-09-10 08:37:36.637",
    "process_timestamp": "20240910_083736637"
}
"""

# Set parameters from "pl_master" in variables

In [ ]:
# Create local variables from the parameters, paseed by "pl_master" into namespace "global_parameter"

global_parameter = j.loads(global_parameter) # Convert string to mapping
global_parameter = sn(**global_parameter)    # Create variables from mapping

if is_debug:
    print(type(global_parameter))
    print(f"global_parameter.days_to_keep_log:                    {global_parameter.days_to_keep_log}")
    print(f"global_parameter.send_success_email:                  {global_parameter.send_success_email}")
    print(f"global_parameter.send_error_email:                    {global_parameter.send_error_email}")
    print(f"global_parameter.email_on_success:                    {global_parameter.email_on_success}")
    print(f"global_parameter.email_on_error:                      {global_parameter.email_on_error}")
    print(f"global_parameter.time_zone_nb:                        {global_parameter.time_zone_nb}")
    print(f"global_parameter.time_zone_pl:                        {global_parameter.time_zone_pl}")
    print(f"global_parameter.mssql_isolation_level:               {global_parameter.mssql_isolation_level}")
    print(f"global_parameter.projet_documentation_data_file_name: {global_parameter.projet_documentation_data_file_name}")
    print(f"global_parameter.abfs_path_lh_log:                    {global_parameter.abfs_path_lh_log}")
    print(f"global_parameter.abfs_path_lh_bronze:                 {global_parameter.abfs_path_lh_bronze}")
    print(f"global_parameter.abfs_path_lh_silver:                 {global_parameter.abfs_path_lh_silver}")
    print(f"global_parameter.abfs_path_lh_cfg:                    {global_parameter.abfs_path_lh_cfg}")
    print(f"global_parameter.my_dataverse:                        {global_parameter.my_dataverse}")
    print(f"global_parameter.now_datetime:                        {global_parameter.now_datetime}")
    print(f"global_parameter.process_timestamp:                   {global_parameter.process_timestamp}")

# Function

In [ ]:
#%run nb_system_cfg

In [ ]:
%run nb_system_function

In [ ]:
# Freeze this
# Reset now_datetime, process_timestamp.

global_parameter.now_datetime      = fn_get_now("datetime")[1]
global_parameter.process_timestamp = fn_get_now("string")[1]

print(f"global_parameter.now_datetime:      {global_parameter.now_datetime}")
print(f"global_parameter.process_timestamp: {global_parameter.process_timestamp}")

In [ ]:
# Freeze this
# Prepare for initial extract

print("clean up lh_cfg.extract_parameter")
display(spark.sql(f"delete from delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_parameter`"))

print("clean up lh_log.extract")
display(spark.sql(f"delete from delta.`{global_parameter.abfs_path_lh_log}/Tables/extract`"))

print("clean up lh_log.log")
display(spark.sql(f"delete from delta.`{global_parameter.abfs_path_lh_log}/Tables/log`"))

In [ ]:
%run nb_report_function

# Email

In [ ]:
exit_value = fn_get_email_report()[2]
if is_debug: print(f"exit_value: {exit_value}")

# Exit

In [ ]:
notebookutils.notebook.exit(exit_value)